In [ ]:
from openai import OpenAI

client = OpenAI()

def chat_with_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": "You are a helpful assistant, well-versed in TRIZ and ARIZ principles."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def user_input(prompt):
    return input(prompt)

def decision_point(question, options):
    print(question)
    for i, option in enumerate(options, start=1):
        print(f"{i}. {option}")
    choice = int(user_input("Select an option: "))
    return options[choice-1]

def generate_ifr(problem_description, budget_limitation, other_limitations, num_ifrs=1):
    prompt = f"Could you please describe the ideal final result (IFR) for the following problem: {problem_description} with the following limitations: Budget limitation is {'none' if not budget_limitation else budget_limitation}, Other limitations: {other_limitations}?"
    return chat_with_gpt(prompt)

# The user inputs their problem and limitations
problem_description = user_input("Describe the problem you are trying to solve: ")
budget_limitation = user_input("Is there a budget limitation? No / Yes, specify the amount: ")
other_limitations = user_input("Describe other limitations (if any): ")
num_ifrs = 1

# Initialize loop control variables
satisfied_with_ifr = False
ifr = ""

while not satisfied_with_ifr:
    ifr = generate_ifr(problem_description, budget_limitation, other_limitations, num_ifrs)
    print("Generated IFR:", ifr)
    
    # Ask if the user is satisfied with the IFR
    decision = decision_point("Are you satisfied with this IFR or would you like to generate another one?", ["I am satisfied with this IFR.", "Generate another IFR."])
    satisfied_with_ifr = decision == "I am satisfied with this IFR."

In [ ]:
def compare_ifr_to_sota(ifr):
    prompt = f"Could you please compare the above IFR with the SOTA, and find out if the solution already exists and is available on the market? IFR: {ifr}"
    return chat_with_gpt(prompt)

def step_back_from_ifr(ifr):
    prompt = f"Could you please take one step back from the IFR, and describe the updated IFR? Current IFR: {ifr}"
    return chat_with_gpt(prompt)

def describe_technical_contradictions(ifr):
    prompt = f"Could you please describe 4 technical contradictions of the ideal final result ({ifr}) and name them 'TC1', 'TC2', 'TC3', 'TC4'?"
    return chat_with_gpt(prompt)

def solve_technical_contradictions(tc_names):
    prompt = f"Could you please use TRIZ standards to solve the technical contradictions ({', '.join(tc_names)}) described earlier?"
    return chat_with_gpt(prompt)

def compare_solution_to_sota(solution):
    prompt = f"Could you please compare the solution that you provided with the state-of-the-art? Solution: {solution}"
    return chat_with_gpt(prompt)

# Step 2: SOTA comparison
sota_comparison = compare_ifr_to_sota(ifr)
print("SOTA Comparison:", sota_comparison)

# Decision point on SOTA comparison
user_decision_sota = decision_point("Is there a solution already available that satisfies the IFR?", ["Yes", "No"])
if user_decision_sota == "Yes":
    print("Since a satisfying solution exists, there might not be a need for further invention.")
else:
    # Step 3: Consider revising IFR if needed
    user_decision_revision = decision_point("Would you like to revise the IFR based on SOTA comparison?", ["Yes", "No"])
    if user_decision_revision == "Yes":
        ifr = step_back_from_ifr(ifr)
        print("Revised IFR:", ifr)
    
    # Step 4: Identify Technical Contradictions
    tc_descriptions = describe_technical_contradictions(ifr)
    print("Technical Contradictions:", tc_descriptions)
    
    # Assuming TC1, TC2, TC3, TC4 are identified for simplicity
    tc_choices = ["TC1", "TC2", "TC3", "TC4", "I don’t like the current TCs, generate new ones please."]
    selected_tcs = decision_point("Select one or more TCs you would like to proceed with (comma-separated for multiple):", tc_choices)
    # Simplified handling; real scenario needs parsing of selected_tcs if multiple are chosen
    
    # Step 6: Solve Technical Contradictions
    solution = solve_technical_contradictions([selected_tcs])  # Adjust this call if multiple TCs are selected
    print("Solution for the Technical Contradictions:", solution)
    
    # Step 7: User decision on the proposed solutions
    user_decision_solution = decision_point("Do you want to proceed with this solution or explore other options?", ["Proceed with this solution.", "Explore other options."])
    if user_decision_solution == "Proceed with this solution.":
        # Step 8: Compare the chosen solution to SOTA
        final_comparison = compare_solution_to_sota(solution)
        print("Final Comparison with SOTA:", final_comparison)
    else:
        print("Exploring other solutions...")
        # Additional logic for exploring other solutions would go here